In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
import gensim
import pickle
import nltk
import copy

In [28]:
### Affect Dimension == emotion angry 0 fear 1 joy 2 sadness 3

In [29]:
### angry  ###
file_angry = pd.read_csv("../data/anger_train.txt",sep="	")
file_angry = file_angry.drop(columns="ID")
f = open("./text_preprocessed_angry.txt", 'w',encoding = 'utf-8')
for i in range(0,len(file_angry["Affect Dimension"])):
    text = ''
    file_angry["Affect Dimension"][i] = 0
    split_text = file_angry["Intensity Class"][i].split(":")
    file_angry["Intensity Class"][i] = split_text[0]
    temp = re.sub('[^a-zA-Z\s@]', '', file_angry["Tweet"][i])
    temp = temp.split(" ")
    temp_list = []
    for j in range(0,len(temp)):
        if "@" in temp[j]:
            temp_list.append("<user>")
            text = text +"<user>"+' '
        elif "" ==temp[j]:
            pass
        else:
            temp_list.append(temp[j].lower())
            text = text + temp[j].lower()+' '
    text=text+'\n'
    f.write(text)
    file_angry["Tweet"][i] = temp_list
f.close()

In [30]:
### fear  ###
file_fear = pd.read_csv("../data/fear_train.txt",sep="	")
file_fear = file_fear.drop(columns="ID")
f = open("./text_preprocessed_fear.txt", 'w',encoding = 'utf-8')
for i in range(0,len(file_fear["Affect Dimension"])):
    text = ''
    file_fear["Affect Dimension"][i] = 1
    split_text = file_fear["Intensity Class"][i].split(":")
    file_fear["Intensity Class"][i] = split_text[0]
    temp = re.sub('[^a-zA-Z\s@#]', '', file_fear["Tweet"][i])
    temp = temp.split(" ")
    temp_list = []
    for j in range(0,len(temp)):
        if "@" in temp[j]:
            temp_list.append("<user>")
            text = text +"<user>"+' '
        elif "#" in temp[j]: #일단 #도 지움
            temp_list.append("<hashtag>")
            text = text +"<hashtag>"+' '
        elif "" ==temp[j]:
            pass
        else:
            temp_list.append(temp[j].lower())
            text = text + temp[j].lower()+' '
    text=text+'\n'
    f.write(text)
    file_fear["Tweet"][i] = temp_list
f.close()

In [31]:
### joy  ###
file_joy = pd.read_csv("../data/joy_train.txt",sep="	")
file_joy = file_joy.drop(columns="ID")
f = open("./text_preprocessed_joy.txt", 'w',encoding = 'utf-8')
for i in range(0,len(file_joy["Affect Dimension"])):
    text = ''
    file_joy["Affect Dimension"][i] = 2
    split_text = file_joy["Intensity Class"][i].split(":")
    file_joy["Intensity Class"][i] = split_text[0]
    temp = re.sub('[^a-zA-Z\s@#]', '', file_joy["Tweet"][i])
    temp = temp.split(" ")
    temp_list = []
    for j in range(0,len(temp)):
        if "@" in temp[j]:
            temp_list.append("<user>")
            text = text +"<user>"+' '
        elif "#" in temp[j]: #일단 #도 지움
            temp_list.append("<hashtag>")
            text = text +"<hashtag>"+' '
        elif "" ==temp[j]:
            pass
        else:
            temp_list.append(temp[j].lower())
            text = text + temp[j].lower()+' '
    text=text+'\n'
    f.write(text)
    file_joy["Tweet"][i] = temp_list
f.close()

In [32]:
### sadness  ###
file_sadness = pd.read_csv("../data/sadness_train.txt",sep="	")
file_sadness = file_sadness.drop(columns="ID")
f = open("./text_preprocessed_sadness.txt", 'w',encoding = 'utf-8')
for i in range(0,len(file_sadness["Affect Dimension"])):
    text = ''
    file_sadness["Affect Dimension"][i] = 3
    split_text = file_sadness["Intensity Class"][i].split(":")
    file_sadness["Intensity Class"][i] = split_text[0]
    temp = re.sub('[^a-zA-Z\s@#]', '', file_sadness["Tweet"][i])
    temp = temp.split(" ")
    temp_list = []
    for j in range(0,len(temp)):
        if "@" in temp[j]:
            temp_list.append("<user>")
            text = text +"<user>"+' '
        elif "#" in temp[j]: #일단 #도 지움
            temp_list.append("<hashtag>")
            text = text +"<hashtag>"+' '
        elif "" ==temp[j]:
            pass
        else:
            temp_list.append(temp[j].lower())
            text = text + temp[j].lower()+' '
    text=text+'\n'
    f.write(text)
    file_sadness["Tweet"][i] = temp_list
f.close()

In [33]:
file = file_angry
# file = pd.concat([file_angry, file_fear],ignore_index=True)
# file = pd.concat([file, file_joy],ignore_index=True)
# file = pd.concat([file, file_sadness],ignore_index=True)

In [34]:
one_hot_affect = file.iloc[:,1:2]
one_hot_intensity = file.iloc[:,2:]

In [35]:
affect = copy.deepcopy(file.iloc[:,1:2])
intensity = copy.deepcopy(file.iloc[:,2:])

In [36]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(one_hot_affect)
one_hot_affect = pd.DataFrame(enc.transform(one_hot_affect).toarray())

In [37]:
enc2 = OneHotEncoder(handle_unknown='ignore')
enc2.fit(one_hot_intensity)
one_hot_intensity = pd.DataFrame(enc2.transform(one_hot_intensity).toarray())

In [38]:
data = pd.concat([file["Tweet"],one_hot_intensity],axis = 1)
#data = pd.concat([data,one_hot_intensity],axis = 1)

In [39]:
sentences_pos = list()
for i in data["Tweet"]:
    tags_en = nltk.pos_tag(i)
    sentences_pos.append(tags_en)

In [40]:
print("Training word2vec ...")
model = gensim.models.Word2Vec(data["Tweet"], size=30, window=5,min_count=0, workers=4, iter=10, sg=1)

Training word2vec ...


In [41]:
num_dic = {n: i for i, n in enumerate(model.wv.vocab)}
dic_num = {i: n for i, n in enumerate(model.wv.vocab)}
num_dic["<start>"] = len(num_dic)
num_dic["<end>"] = len(num_dic)
num_dic["<unknown>"] = len(num_dic)
dic_num[len(dic_num)] = "<start>"
dic_num[len(dic_num)] = "<end>"
dic_num[len(dic_num)] = "<unknown>"

In [42]:
input_batch = []
output_batch = []
for word in data["Tweet"]:
    # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
    input = [num_dic[n] for n in word]
    input_batch.append(input)
    # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
    output = [num_dic["<start>"]]
    for i in word:
        output.append(num_dic[i])
    output_batch.append(output)

In [43]:
# word2vec에 <start>, UNK 등 추가 후 numpy로 저장
key = list(num_dic.keys())
w2v = []
for k in key:
    if k == '<start>' or k == '<unknown>' or k == "<end>":
        w2v.append(np.random.randn(30)*0.1)
    else:
        w2v.append(model.wv[k])

In [44]:
data.to_csv("../data/preprocessed_data.csv",sep = ",",index = False)
with open('sentence.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
with open('sentence_pos.pickle', 'wb') as f:
    pickle.dump(sentences_pos, f, pickle.HIGHEST_PROTOCOL)
with open('sentence_real_data_input.pickle', 'wb') as f:
    pickle.dump(input_batch, f, pickle.HIGHEST_PROTOCOL)
with open('sentence_real_data_add_start_end.pickle', 'wb') as f:
    pickle.dump(output_batch, f, pickle.HIGHEST_PROTOCOL)
with open('num_dic.pickle', 'wb') as f:
    pickle.dump(num_dic, f, pickle.HIGHEST_PROTOCOL)
with open('dic_num.pickle', 'wb') as f:
    pickle.dump(dic_num, f, pickle.HIGHEST_PROTOCOL)
with open('word_embedding_matrix.pickle', 'wb') as f:
    pickle.dump(w2v, f, pickle.HIGHEST_PROTOCOL)
#### 감정 세기 데이터에서 1이 원본데이터에서의 0에 해당됨.

In [45]:
file = open('./before_train_text.txt', 'w',encoding = 'utf-8')
for idx,sentence in enumerate(data["Tweet"]):
    for j in sentence:
        file.write(j+' ')
    file.write('\n')
file.close()

In [46]:
model.save('model')

In [47]:
file = open('../model/save/positive_file.txt', 'w')
for idx,sentence in enumerate(input_batch):
    #file.write(str(affect["Affect Dimension"][idx])+' ')
    file.write(str(intensity["Intensity Class"][idx])+' ')
    for word in sentence:
        word = str(word) + ' '
        file.write(word)
    if len(sentence)<40:
        for _ in range(0,40-len(sentence)):
            file.write('4421 ')
    file.write('\n')
file.close()

In [48]:
# 다 학습시켜도 되려나? 일단 뭐... 학습해보라해

In [49]:
file_angry.head()

,Tweet,Affect Dimension,Intensity Class
0,"[<user>, <user>, shut, up, hashtags, are, cool...",0,2
1,"[it, makes, me, so, fucking, irate, jesus, nob...",0,3
2,"[lol, adam, the, bull, with, his, fake, outrage]",0,1
3,"[<user>, passed, away, early, this, morning, i...",0,0
4,"[<user>, lol, wow, i, was, gonna, say, really,...",0,1
